In [94]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sys
import json

# Adicione apenas a pasta, não o arquivo .py
sys.path.append(r'C:\Users\USER\Desktop\GitHub\Brasileirao-Analise\3. Source Code\Transformed')
import datadetox as detox


In [ ]:
# URL da página
url = "https://www.transfermarkt.com.br/campeonato-brasileiro-serie-a/besucherzahlen/wettbewerb/BRA1/plus/1?saison_id="

# Cabeçalhos para simular um navegador
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36"}

df = pd.DataFrame(columns=["Estadio", "Clube", "Capacidade", "Publico Anual", "Media", "Num Jogos"])

for season in range (2002,2024):
    url_season = url+str(season)
    # Requisição HTTP
    response = requests.get(url_season, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Tabela com os dados de público
    table = soup.find("table", class_="items")

    # Extração de linhas
    rows = table.find_all("tr", class_=["odd", "even"])

    data = []
    for row in rows:
        cols = row.find_all("td")
        if len(cols) >= 6:
            estadio = cols[3].text.strip()
            clube = cols[4].text.strip().replace('.', '').replace('-', '0')
            capacidade = cols[5].text.strip().replace('.', '').replace('-', '0')
            publico_anual = cols[6].text.strip().replace('.', '').replace('-', '0')
            media = cols[7].text.strip().replace('.', '').replace('-', '0')
            num_jogos = cols[8].text.strip().replace('%', '').replace('-', '0')
            data.append([estadio,clube,int(capacidade),int(publico_anual),int(media),int(num_jogos)])

    # Criar DataFrame
    df_aux = pd.DataFrame(data, columns=["Estadio", "Clube", "Capacidade", "Publico Anual", "Media", "Num Jogos"])
    df_aux['Temporada'] = season+1
    # Exibir amostra
    df = pd.concat([df,df_aux],ignore_index=True)


df['Estadio'] = df['Estadio'].apply(detox.normalizar_texto)
df['Estadio'] = df['Estadio'].str.replace(r'\bestadio\b', '', regex=True).str.strip()
df['Estadio'] = df['Estadio'].str.replace(r'\(\*pf\)', '', regex=True).str.strip()
df['Estadio'] = df['Estadio'].str.replace(r'\*\(pf\)', '', regex=True)

mapeamento_estadios = detox.ler_json_as_dict(r'C:\Users\USER\Desktop\GitHub\Brasileirao-Analise\3. Source Code\Transformed\arenas_dict.json')
df['Estadio'] = df['Estadio'].replace(mapeamento_estadios)


# #Salvar em CSV
# df_final.to_excel("publico_brasileirao.xlsx", index=False)

dfgroup = df.groupby(by='Estadio',dropna=True)['Capacidade']

maximos_por_estadio = dfgroup.min().to_dict()
with open("estadios.json", "w", encoding="utf-8") as f:
    json.dump(maximos_por_estadio, f, ensure_ascii=False, indent=2)
